# multivariate 
# univariate 


In [57]:
from statsmodels.tsa.vector_ar.var_model import VAR
from statsmodels.tsa.statespace.sarimax import SARIMAX

import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [15]:
df = pd.read_csv("data/daily_funnel_kpis.csv")

In [16]:
df.head()

Date  Cart Additions  Checkout Visit (e43) (event43)  \
0  January 1, 2018             221                               0   
1  January 2, 2018            9550                            3637   
2  January 3, 2018           11611                            4680   
3  January 4, 2018           13124                            5243   
4  January 5, 2018           11098                            4263   

   Start Checkout (e71) (event71)  Orders    Revenue  Unique Visitors  Visits  
0                               0    2588   24174.85            26065   32293  
1                            5867    6082  502260.02           124011  144365  
2                            7364    6475  624929.47           150293  176357  
3                            9134    7830  514532.75           152939  183323  
4                            7022    5019  574056.18           128820  151070

In [21]:
df['Date']=pd.to_datetime(df['Date'], dayfirst=True)

In [22]:
df.head()

Date  Cart Additions  Checkout Visit (e43) (event43)  \
0 2018-01-01             221                               0   
1 2018-01-02            9550                            3637   
2 2018-01-03           11611                            4680   
3 2018-01-04           13124                            5243   
4 2018-01-05           11098                            4263   

   Start Checkout (e71) (event71)  Orders    Revenue  Unique Visitors  Visits  
0                               0    2588   24174.85            26065   32293  
1                            5867    6082  502260.02           124011  144365  
2                            7364    6475  624929.47           150293  176357  
3                            9134    7830  514532.75           152939  183323  
4                            7022    5019  574056.18           128820  151070

In [34]:
df.index = df.Date
df.drop('Date', axis=1, inplace=True)

In [35]:
df.head()

Cart Additions  Checkout Visit (e43) (event43)  \
Date                                                         
2018-01-01             221                               0   
2018-01-02            9550                            3637   
2018-01-03           11611                            4680   
2018-01-04           13124                            5243   
2018-01-05           11098                            4263   

            Start Checkout (e71) (event71)  Orders    Revenue  \
Date                                                            
2018-01-01                               0    2588   24174.85   
2018-01-02                            5867    6082  502260.02   
2018-01-03                            7364    6475  624929.47   
2018-01-04                            9134    7830  514532.75   
2018-01-05                            7022    5019  574056.18   

            Unique Visitors  Visits  
Date                                 
2018-01-01            26065   32293  
2018-01-02           124011  144365  
2018-01-03           150293  176357  
2018-01-04           152939  183323  
2018-01-05           128820  151070

## multivariate 

In [36]:
from statsmodels.tsa.vector_ar.vecm import coint_johansen
#since the test works for only 12 variables, I have randomly dropped
#in the next iteration, I would drop another and check the eigenvalues
johan_test_temp = df.drop(['Revenue'], axis=1)
johan_test_temp.head()
coint_johansen(johan_test_temp,-1,1).eig

array([0.23809549, 0.17665887, 0.10645891, 0.07420832, 0.0340283 ,
       0.00429375])

In [38]:
#creating the train and validation set
train = df[:int(0.8*(len(df)))]
valid = df[int(0.8*(len(df))):]

#fit the model
from statsmodels.tsa.vector_ar.var_model import VAR

model = VAR(endog=train)
model_fit = model.fit()

# make prediction on validation
prediction = model_fit.forecast(model_fit.y, steps=len(valid))

/Users/sulbha/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)


In [65]:
len(valid)

74

In [44]:
cols =df.columns

In [53]:
#converting predictions to dataframe
import numpy as np
import sklearn.metrics as m


pred = pd.DataFrame(index=range(0,len(prediction)),columns=[cols])


for j in range(0,7):
    for i in range(0, len(prediction)):
       pred.iloc[i][j] = prediction[i][j]

#check rmse
for i in cols:
    print('rmse value for', i, 'is : ', np.sqrt(m.mean_squared_error(pred[i], valid[i])))

rmse value for Cart Additions is :  14873.963930091124
rmse value for Checkout Visit (e43) (event43) is :  3275.7395429676662
rmse value for Start Checkout (e71) (event71) is :  8137.485500458355
rmse value for Orders is :  3206.2565458994563
rmse value for Revenue is :  2098223.0589394136
rmse value for Unique Visitors is :  29783.112332258024
rmse value for Visits is :  39971.940830916814


In [63]:
pred

Cart Additions Checkout Visit (e43) (event43)  \
0         16410.6                        4381.64   
1         15992.4                        4435.02   
2         15675.1                        4480.21   
3         15421.9                        4514.79   
4           15213                        4541.01   
5         15036.7                        4561.24   
6         14885.5                        4577.24   
7         14754.2                        4590.19   
8         14639.1                        4600.87   
9         14537.5                        4609.83   
10        14447.2                        4617.45   
11        14366.7                        4623.98   
12        14294.4                        4629.63   
13        14229.5                        4634.55   
14          14171                        4638.86   
15        14118.1                        4642.65   
16        14070.3                        4646.01   
17        14027.1                        4648.98   
18        13987.8                        4651.62   
19        13952.3                        4653.98   
20          13920                        4656.09   
21        13890.6                        4657.97   
22          13864                        4659.67   
23        13839.8                         4661.2   
24        13817.7                        4662.57   
25        13797.7                        4663.81   
26        13779.5                        4664.94   
27        13762.9                        4665.95   
28        13747.8                        4666.87   
29        13734.1                         4667.7   
..            ...                            ...   
44        13628.8                        4673.97   
45        13625.7                        4674.16   
46          13623                        4674.32   
47        13620.4                        4674.47   
48        13618.1                         4674.6   
49          13616                        4674.73   
50        13614.1                        4674.84   
51        13612.3                        4674.94   
52        13610.7                        4675.04   
53        13609.3                        4675.12   
54          13608                         4675.2   
55        13606.8                        4675.27   
56        13605.7                        4675.34   
57        13604.7                        4675.39   
58        13603.8                        4675.45   
59          13603                         4675.5   
60        13602.2                        4675.54   
61        13601.5                        4675.58   
62        13600.9                        4675.62   
63        13600.3                        4675.65   
64        13599.8                        4675.68   
65        13599.3                        4675.71   
66        13598.9                        4675.73   
67        13598.5                        4675.76   
68        13598.2                        4675.78   
69        13597.8                         4675.8   
70        13597.5                        4675.81   
71        13597.3                        4675.83   
72          13597                        4675.84   
73        13596.8                        4675.86   

   Start Checkout (e71) (event71)   Orders Revenue Unique Visitors  Visits  
0                         7629.34  4736.88  586316          120548  146081  
1                         7707.84  4844.95  588107          123288  150412  
2                         7735.48  4960.81  588570          124883  152834  
3                         7754.65  5065.76  588429          125883  154237  
4                         7772.35     5155  588502          126580  155108  
5                         7788.82  5228.62  589017          127122  155700  
6                         7803.54  5288.29  589901          127583  156145  
7                         7816.27  5336.12  591004          127997  156508  
8                         7827.01  5374.12  592195          128378  156823  
9       

In [56]:
#make final predictions
model = VAR(endog=df)
model_fit = model.fit()
yhat = model_fit.forecast(model_fit.y, steps=12)
print(yhat)

[[ 13765.66089216   4530.7940116    6449.63589025   6055.55311993
  496829.11478513 121255.55128971 144292.29377745]
 [ 14341.30627803   4664.92897712   6945.91832045   5949.53505576
  583426.54409341 125838.40650615 150400.22443366]
 [ 14722.86231818   4756.84481643   7297.23350446   5871.32365849
  633696.00192772 128729.0086169  154268.45718093]
 [ 14971.29396911   4815.8600601    7548.7958142    5812.48339912
  665772.18245161 130524.9834218  156687.7739216 ]
 [ 15133.9893973    4853.88360827   7730.93350675   5768.78193127
  687070.26655765 131649.66627796 158217.14626536]
 [ 15241.26037604   4878.81858047   7863.89659477   5736.81667352
  701471.27699512 132363.54339792 159200.04886824]
 [ 15312.04785986   4895.44109295   7961.49896786   5713.74037662
  711288.9392947  132822.87524507 159842.42819069]
 [ 15358.3915354    4906.64039654   8033.38963688   5697.2702491
  717994.0787747  133121.91164779 160268.60440664]
 [ 15388.14501924   4914.21341641   8086.4411636    5685.64255871

/Users/sulbha/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)


# univariate - SARIMA

In [62]:
from statsmodels.tsa.statespace.sarimax import SARIMAX


model = SARIMAX(train['Cart Additions'], order = (1,1,1), seasonal_order=(1,1,1,1))

model_fit = model.fit(disp=False)

y_hat = model_fit.predict(len(valid), len(valid))
print (y_hat)

/Users/sulbha/anaconda3/lib/python3.7/site-packages/statsmodels/tsa/base/tsa_model.py:165: ValueWarning: No frequency information was provided, so inferred frequency D will be used.
  % freq, ValueWarning)


2018-03-16    16095.407891
Freq: D, dtype: float64


In [66]:
diff = (16410.6-16095.407891)/16095.407891
diff

0.019582735096526655